In [28]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

from tensorflow import keras


rng = 69

In [29]:
#df = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_WLT_Transformed.csv")
df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_WLT_Transformed.csv")

In [30]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_loss_tie_Away'])

In [31]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [32]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [33]:
#corr_matrix['win_Home'].sort_values(ascending=False)

KeyError: 'win_Home'

In [ ]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [34]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [35]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('win_loss_tie_Home', axis=1)
nhl_labels = train_set['win_loss_tie_Home'].copy()

X_train = train_set.drop('win_loss_tie_Home', axis=1)
Y_train = train_set['win_loss_tie_Home'].copy()

X_valid = train_set.drop('win_loss_tie_Home', axis=1)
Y_valid = train_set['win_loss_tie_Home'].copy()

X_test = test_set.drop('win_loss_tie_Home', axis=1)
Y_test = test_set['win_loss_tie_Home'].copy()

df_Final = df_Final.drop('win_loss_tie_Home', axis=1)

In [36]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [45]:
print(Y_train)

4042    0
3670    0
1614    0
1423    2
1530    0
       ..
3633    0
439     2
1626    0
2667    1
4041    2
Name: win_loss_tie_Home, Length: 3496, dtype: int64


In [37]:
# set epochs to use same num in all calls
n_epochs = 40

In [46]:
def build_model(learning_rate = 0.00335195324238683, n_hidden=1, n_neurons=15, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    if n_hidden > 0:
        for _ in range(n_hidden):
            model.add(keras.layers.Dense(n_neurons,activation='selu',kernel_initializer="lecun_normal"))
    model.add(keras.layers.Dense(3, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer= keras.optimizers.Adam(learning_rate=learning_rate))
    return model

In [47]:
# Build model using best params from hypertuning

model = build_model

In [48]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_multi_clf = KerasClassifier(model=model, epochs=n_epochs, batch_size=5, verbose=0, random_state=42,
                           loss='sparse_categorical_crossentropy',optimizer='adam',metrics='accuracy')

# Stop early if model is not getting better after # of patience epochs, restore to best model
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
#checkpoint_cb = keras.callbacks.ModelCheckpoint("Regression_Sequential.keras", save_best_only = True)
checkpoint_cb = keras.callbacks.ModelCheckpoint(save_file, save_best_only = True)

keras_multi_clf.fit(X_train_prepared, Y_train, 
              validation_data = (X_valid_prepared, Y_valid),
              callbacks=[early_stopping_cb,checkpoint_cb])



KerasClassifier(
	model=<function build_model at 0x000001F2B52D0C20>
	build_fn=None
	warm_start=False
	random_state=42
	optimizer=adam
	loss=sparse_categorical_crossentropy
	metrics=accuracy
	batch_size=5
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=40
	class_weight=None
)

In [52]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold


kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
#Y_pred = cross_val_predict(keras_clf, X_train_prepared, Y_train, cv=kfold)

Y_pred = cross_val_score(keras_multi_clf, X_train_prepared, Y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 54.09% (0.94%)


In [55]:
# Train Accuracy
Y_train_pred = keras_multi_clf.predict(X_train_prepared)
print(accuracy_score(Y_train, Y_train_pred))

Y_test_pred = keras_multi_clf.predict(X_test_prepared)
print(accuracy_score(Y_test, Y_test_pred))

0.5643592677345538
0.5394285714285715


In [59]:
#df_Predict = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_WLT_Predict.csv")
df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_WLT_Predict.csv")

In [65]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_multi_clf.predict(X_real_predict))).T

array([['TBL', 0],
       ['DET', 2],
       ['PIT', 2],
       ['NJD', 2]], dtype=object)

,seasonPointsPerGame_Home,goalDiffAvg_Home,hitsDiffAvg_Home,pointsFromGameAvg_Home,fenwickPercentageAvg_Home,reboundsForAvg_Home,seasonPointsPerGame_Away,goalDiffAvg_Away,hitsDiffAvg_Away,pointsFromGameAvg_Away,fenwickPercentageAvg_Away,reboundsForAvg_Away,penaltiesForTotal,penaltiesAgainstTotal
0,1.075000,1.14,-1.02,1.28,0.48,2.44,0.925926,0.44,-4.42,1.07,0.47,2.15,6.90,7.73
1,0.962500,-0.34,0.98,0.55,0.50,3.18,0.787500,0.17,0.46,0.64,0.40,2.09,5.53,7.70
2,0.937500,0.95,-3.69,1.47,0.50,4.96,1.123457,0.71,-1.12,0.99,0.57,4.76,7.56,6.97
3,0.962963,-0.42,-5.45,1.06,0.41,1.63,0.975000,1.13,0.74,1.82,0.55,3.48,7.48,6.07
